In [1]:
#_*_encoding:utf-8_*_
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests, json
import random
import pandas as pd
import matplotlib.pyplot as plt
import re


## 方法一：使用程式打開瀏覽器

In [2]:
chromeDriver = './chromedriver_mac64/chromedriver'
service_obj = Service(chromeDriver)
driver = webdriver.Chrome(service=service_obj)
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36', }

TypeError: __init__() got an unexpected keyword argument 'service'

## 方法二：使用已開啟瀏覽器（需驗證登入時）
終端機輸入 
1. export PATH="/Applications/Google Chrome.app/Contents/MacOS:$PATH"
2. Google\ Chrome --remote-debugging-port=9222 --user-data-dir="~/ChromeProfile"
開啟瀏覽器

In [2]:
chromeDriver = 'C:/D\中山/31lab/裕隆日產/chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
driver = webdriver.Chrome(executable_path=chromeDriver, options=options)
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36', }

## 開啟瀏覽器

In [2]:
driver = webdriver.Chrome('C:/D\中山/31lab/裕隆日產/chromedriver.exe')

In [3]:
driver.close()

In [5]:
driver = webdriver.Chrome('../chromedriver')
# driver.get("https://mbasic.facebook.com/groups/internlens/?refid=46&__xts__%5B0%5D=12.Abpf2d6pkijgmhRLt7n7t1TRb5vDnFWgmb0EL4XLVvTqn6ac0M8i3OCL1uXgyUAqDmUQ1prkYzWTRC00J5wkfJ2QShJoh8PmcfM5lSlO4gGVW_yeUAF6SAzsb16HeC35Cpp-GlHhZLdxM_nez3BTyy8fpzXBc5clx7CFbTccLJYnzJoGRZCbG32pZRnTgjXJDpixDkKdMCEKHwGkqa3jBWXQmuCNp5L4auZAIR02UEmT8WaVi5llQt-ut2GFNqDf4NRJ0unf73y9v0Ah6rVAwV_NzIG47oH1vIHeG88awtCPJAenpOz8fpbS_IXrvivUufX7oY7AB0gqeZvp_qlPp-AWjKrap4NSJYQRq8_HTEJPztepiAQ5tmiTS8Y-NkLiYCfo-oi7qbeJQ9sWmIEHl1AtHe8t7piv3ckq3x_oDa7ZkTbl8e0bYS-a-RS1wxYn1Z7Tr0XLSYyn446cqQj9GbJ8NVRvsWdNpbiIQJzwrNAjzuixSjcaWBwhik7ao1xJ5A1GklNd-Wu49SAnek9QuuKRcHw2u0j1oPoPrmyIHZfx4z8HohwHQ3sb1X34Cr566fQ&paipv=0&eav=AfbFF1e-i2bOSDdpcF-pr4g_4urOZ59CMpmhF99mlnN9lPQJ4uoCl9ilr_PTnFEs4yQ")
# print(driver.title)

In [110]:
# 測試
driver.get("https://salary.tw/auth/login?redirect=https%3A%2F%2Finterview.tw")

## 登入

In [5]:
# User_Agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
# opt = webdriver.ChromeOptions()
# opt.add_argument('--user-agent=%s' % User_Agent)
# driver = webdriver.Chrome('../chromedriver',options=opt)
driver = webdriver.Chrome('../chromedriver')
driver.get("https://salary.tw/auth/login?redirect=https%3A%2F%2Finterview.tw")
time.sleep(2)
button = driver.find_element(By.PARTIAL_LINK_TEXT, "使用 Google 繼續")
button.click()
time.sleep(2)
account_text = driver.find_element(By.XPATH, '//*[@id="identifierId"]')
time.sleep(2)
account_text.send_keys('ynmnsysu@gmail.com') # 輸入帳號
login_btn = driver.find_element(By.XPATH, '//*[@id="identifierNext"]/div/button/span')
login_btn.click()
time.sleep(2)
password_text = driver.find_element(By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')
time.sleep(2)
password_text.send_keys('Ynmnsysu2023')  # 輸入密碼
login_btn_2 = driver.find_element(By.XPATH, '//*[@id="passwordNext"]/div/button/span')
login_btn_2.click()

## 抓取比薪水資料

In [3]:
companies = [
"台灣電力股份有限公司:8ijc",
"鴻海精密工業股份有限公司:3Nan",
"國泰世華商業銀行股份有限公司:U3dQ",
"台北富邦商業銀行股份有限公司:9WIp",
"台灣中油股份有限公司:W3DS",
"奧美廣告股份有限公司:xhjc",
"日立家電 (台灣)股份有限公司:clxK",
"台灣日立江森自控股份有限公司（原：台灣日立）:BR91",
"中國信託金融控股股份有限公司:MqlG",
"COSTCO_好市多股份有限公司:ZOC7",
"裕隆集團_納智捷汽車股份有限公司:aQUw",
"中華電信股份有限公司:528F",
"台灣大哥大股份有限公司:olVI",
"臺灣塑膠工業股份有限公司:W81V",
"緯創資通股份有限公司:wcWn",
"緯創軟體股份有限公司:ayUp",
"元大金融控股股份有限公司 (元大金控) (元大證券) (元大銀行):Dfrt",
"全聯福利中心_全聯實業股份有限公司:VjhE",
"PChome Online_ 網路家庭國際資訊股份有限公司:LAQY",
"聯詠科技股份有限公司:s6XL",
"陽明資訊股份有限公司:xpVu",
"台新國際商業銀行股份有限公司:KDQa",
"台新金融控股股份有限公司:nTAL",
"大立光電股份有限公司:U9fO",
"鴻海精密工業股份有限公司:3Nan",
"中國鋼鐵股份有限公司:cyYa",
"(ASML)台灣艾司摩爾科技股份有限公司:C7yj",
"聯發科通訊股份有限公司:48Gz",
"MTK_聯發科技股份有限公司:JZmq",
"台達電子工業股份有限公司_DELTA ELECTRONICS INC.:ZWBD",
"聯電_聯華電子股份有限公司:L4z2",
"LITEON_光寶科技股份有限公司:CUac",
"Quanta Computer_廣達電腦股份有限公司:xOGz",
"上銀科技股份有限公司:4iHv",
"NCSIST 中科院_國家中山科學研究院:sFhw",
"日月光半導體製造股份有限公司（高雄廠）:QpfM",
"日月光半導體製造股份有限公司中壢分公司:pjeC",
"美商科磊股份有限公司:wf9Q",
"COMPAL_仁寶電腦工業股份有限公司:KQr0",
"交通部臺灣鐵路管理局:eXGu",
"強淞企業股份有限公司:29cg",
"漢翔航空工業股份有限公司:k5Eh",
"Unimicron_欣興電子股份有限公司:OTVU",
"ASUS_華碩電腦股份有限公司:iCka",
"IBM_台灣國際商業機器股份有限公司:MbYr",
"美商蘋果亞洲股份有限公司:HaRb",
"華映科技有限公司:s78h",
"Phison Electronics Corp_群聯電子股份有限公司:rpo3",
"穩懋半導體股份有限公司:Yxep",
"Qisda_佳世達科技股份有限公司:gutG",
"長春石油化學股份有限公司:hKDs"]

In [4]:
# 設置要爬取的公司代碼, xrbe 後為科技公司
# aQUw:納智捷, xmWa:北智捷, tjzM:中智捷, 0U2N:裕隆汽車, AfWZ:裕隆日產, lwNQ:中華汽車, W139:三陽工業, MGal:福特六和, kchg:國瑞汽車, mU04:台灣本田, i9Um:成運汽車, gWc5:華德動能, 6b50C:鴻華先進, YFgq:為升電裝, xrbe:美光, UDCP:康寧, hKDs:長春石化, 2Iqp:友達光電, l8Yc:捷安特, 2NWa:力積電, 6lU8:華邦電子, U9fO:大立光, hdAT:寶成工業, KYRX:台積電, J7Va:格上租車, NmC3:裕融企業, 2bnH:和泰汽車, cUN9:中華賓士, nYCR:汎德永業, HYto:太古汽車, jT7Q:匯豐汽車
company_code = ['aQUw', 'xmWa', 'tjzM', '0U2N', 'AfWZ', 'lwNQ', 'W139', 'MGal', 'kchg', 'mU04', 'i9Um', 'gWc5', '6b50C', 'YFgq', 'xrbe', 'UDCP', 'hKDs', '2Iqp', 'l8Yc', '2NWa', '6lU8', 'U9fO', 'hdAT', 'KYRX', 'J7Va', 'NmC3', '2bnH', 'cUN9', 'nYCR', 'HYto', 'jT7Q']
Newcompany_code = [i.split(":")[1] for i in companies]
company_code += Newcompany_code

In [5]:
len(company_code)

82

In [15]:
# # 存放每間公司爬取資訊
df_interview = pd.DataFrame(columns=['company_name','company_code','industry', 'location', 'mood', 'frequence', 'salary_month', 'interview_num', 'salary_num'])
# 不同公司
for c in company_code:
    try:
        # 不同頁面
        time.sleep(2)
        url = f'https://salary.tw/c/{c}?sort=newest'
        driver.get(url)
        resp = driver.page_source
        soup = BeautifulSoup(resp,"html.parser")
        
        # 公司名稱
        _company_name = soup.find('h1', {'class': 'fz-h2 fz-tit'}).text
        _location = soup.find('div', {'class': 'sa_info-list'}).find('div', {'class': 'sa_city-item'}).find('span', {'class': 'fz-s3'}).text
        _industry = soup.find('ul', {'class': 'sa_breadcrumb-wrap'}).find_all('li')[-2].find('a').text
        info_list = soup.find('div', {'class': 'sa_company-info-list'}).find_all('div', {'class':'sa_main-info'})
        info_list = [i.find("span").text for i in info_list]
        _mood = info_list[0]
        _frequence = info_list[1]
        _salary_m = info_list[2]
        _interviewNum = info_list[3]
        _salary_num = info_list[4]
        tmp_df_interview = pd.DataFrame({
            "company_name": [_company_name],
            "company_code":[str(c)],
            "industry":[_industry],
            "location":[_location],
            "mood":[_mood],
            "frequence":[_frequence],
            "salary_month":[_salary_m],
            "interview_num":[_interviewNum],
            "salary_num":[_salary_num]
        }, index=[len(df_interview)])
        df_interview = df_interview.append(tmp_df_interview)
    except:
        driver.refresh()

In [16]:
df_interview

,company_name,company_code,industry,location,mood,frequence,salary_month,interview_num,salary_num
0,裕隆集團_納智捷汽車股份有限公司,aQUw,其他,苗栗,平常心,偶爾加班,39k,18,20
1,Luxgen_北智捷汽車股份有限公司,xmWa,汽機車及其零配件用品／批發業,臺北,平常心,偶爾加班,39.8k,6,7
2,LUXGEN納智捷汽車_中智捷汽車股份有限公司,tjzM,其他,臺中,有苦說不出,常常加班,34.1k,6,13
3,裕隆集團_裕隆汽車製造股份有限公司,0U2N,汽車及其零件製造業,苗栗,有苦說不出,偶爾加班,41.9k,90,136
4,裕隆集團_裕隆日產汽車股份有限公司,AfWZ,汽車及其零件製造業,苗栗,平常心,偶爾加班,43.9k,36,88
...,...,...,...,...,...,...,...,...,...
77,華映科技有限公司,s78h,其他,臺中,還算愉快,很少加班,47.7k,10,2
78,Phison Electronics Corp_群聯電子股份有限公司,rpo3,IC設計相關業,新竹,平常心,常常加班,78.3k,227,473
79,穩懋半導體股份有限公司,Yxep,半導體製造業,桃園,平常心,偶爾加班,45.2k,192,230
80,Qisda_佳世達科技股份有限公司,gutG,電腦及其週邊設備製造業,桃園,平常心,偶爾加班,52.5k,200,180


In [17]:
df_c = df_interview[['company_name','industry','location', 'mood', 'frequence', 'salary_month', 'interview_num', 'salary_num']]
df_c.to_csv('./company_info.csv', encoding='utf-8-sig')

In [12]:
url = f'https://salary.tw/c/aQUw?currentPage=2&sort=newest'
driver.get(url)
# 獲取網頁內容
pageSource = driver.page_source
soup = BeautifulSoup(pageSource, 'html.parser')
# len(soup.find_all('div', {'class':'sa_salary-card'}))
soup.find('h1', {'class': 'fz-h2 fz-tit'}).text
info_list = soup.find('div', {'class': 'sa_company-info-list'}).find_all('div', {'class':'sa_main-info'})
info_list = [i.find("span").text for i in info_list]
info_list

['平常心', '偶爾加班', '39k', '18', '20']

In [14]:
soup.find('ul', {'class': 'sa_breadcrumb-wrap'}).find_all('li')[-2].find('a').text
soup.find('div', {'class': 'sa_info-list'}).find('div', {'class': 'sa_city-item'}).find('span', {'class': 'fz-s3'}).text

'苗栗'

In [7]:
# 不同公司
result = []
for c in company_code:
    try:
        time.sleep(2)
        url = f'https://salary.tw/c/{c}?sort=newest'
        driver.get(url)
        resp = driver.page_source
        soup = BeautifulSoup(resp,"html.parser")
        # 獲取總頁數
        ul = soup.find_all('ul', class_='pagination')  
        # totalpages = int(ul[-1].find_all("li")[-1].find("span").text)
        totalpages = int(ul[-1].find_all("li")[-1].find("span").text) if len(ul) != 0 else 1
        print(c,':',totalpages)
        for page_num in range(1, totalpages+1):
            url = f'https://salary.tw/c/{c}?currentPage={page_num}&sort=newest'
            driver.get(url)
            # 獲取網頁內容
            pageSource = driver.page_source
            soup = BeautifulSoup(pageSource, 'html.parser')
            # 解鎖vip文章
            vip_buttons = driver.find_elements_by_class_name("btn-exchange-vip")
            for button in vip_buttons:
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'auto', block: 'center', inline: 'center'});", button)
                button.click()
                time.sleep(2)
            # 公司名稱
            _company_name = soup.find('h1', {'class': 'fz-h2 fz-tit'}).text
            content = (soup.find_all('div', {'class':'sa_salary-card'}))
            for i in range(len(content)):
                #職位
                _vacancies = content[i].find('a', class_ = 'fz-s1 fz-tit').text
                #分享時間
                _post_time = content[i].find_all('span', {'class':'fz-s4 cl-basic-500'})[1].text[:-3]
                #工作性質
                _job = content[i].find_all('span', {'class':'fz-s4 cl-salary-500'})[0].text
                #地點
                _location = content[i].find_all('span', {'class':'fz-s4 cl-basic-500'})[0].text
                #月薪
                meta_data = content[i].find_all('span', class_ = 'fz-t3 fz-tit')
                _m_salary = meta_data[0].text
                #年薪
                _y_salary = meta_data[1].text
                #年終
                _ye_salary = meta_data[2].text.strip()
                #三節
                _bonus = meta_data[3].text.strip()
                #分紅
                _dividends = meta_data[4].text.strip()
                #其他
                _other = meta_data[5].text.strip()
                #上班心情
                _mood = meta_data[6].text.strip()
                #加班頻率
                _f_overtime = meta_data[7].text.strip()
                #日均工時
                _average_day = meta_data[8].text.strip()
                #年資
                s_list = re.findall(r'\d+', meta_data[9].text.strip())
                _seniority = int(s_list[0]) if len(s_list) != 0 else 0
                #總年資
                st_list = re.findall(r'\d+', meta_data[10].text.strip())
                _seniority_total = int(st_list[0]) if len(st_list) != 0 else 0
                #工作內容
                content_list = content[i].select('span.fz-t3:not([class*=" "])')
                _w_content = content_list[0].text if len(content_list) != 0 else ''
                #其他建議
                _w_advise = content_list[1].text if len(content_list) >= 2 else ''
                post_detail = {
                    'vacancies' : _vacancies,
                    'company' : _company_name,
                    'post_time' : _post_time,
                    'job' : _job,
                    'location' : _location,
                    'month_salary' : _m_salary if _m_salary != '兌換後顯示' else None,
                    'year_salary' : _y_salary if _y_salary != '兌換後顯示' else None,
                    'year-end_salary' :  None if (_ye_salary == '?' or _ye_salary =='-') else _ye_salary,
                    'bonus' : None if (_bonus == '?' or _bonus =='-') else _bonus,
                    'dividends' : None if (_dividends == '?' or _dividends =='-') else _dividends,
                    'other' : None if (_other == '?' or _other =='-') else _other,
                    'mood' : _mood,
                    'frequency_overtime' : _f_overtime,
                    'average_day' : _average_day,
                    'seniority' : _seniority,
                    'total_seniority' : _seniority_total,
                    'work_content' : _w_content,
                    'advise_content' : _w_advise
                }
                result.append(post_detail)
    except:
        driver.refresh()


aQUw : 2
xmWa : 1
tjzM : 1
0U2N : 6
AfWZ : 3
lwNQ : 8
W139 : 7
MGal : 4
kchg : 5
mU04 : 2
i9Um : 1
gWc5 : 1
6b50C : 1
YFgq : 3
xrbe : 46
UDCP : 14
hKDs : 6
2Iqp : 73
l8Yc : 5
2NWa : 2
6lU8 : 14
U9fO : 13
hdAT : 25
KYRX : 146
J7Va : 2
NmC3 : 2
2bnH : 4
cUN9 : 1
nYCR : 4
HYto : 3
jT7Q : 4
8ijc : 10
3Nan : 61
U3dQ : 30
9WIp : 15
W3DS : 17
xhjc : 10
clxK : 1
BR91 : 7
MqlG : 10
ZOC7 : 10
aQUw : 2
528F : 16
olVI : 30
W81V : 20
wcWn : 78
ayUp : 13
Dfrt : 11
VjhE : 30
LAQY : 21
s6XL : 12
xpVu : 1
KDQa : 30
nTAL : 1
U9fO : 13
3Nan : 61
cyYa : 9
C7yj : 18
48Gz : 1
JZmq : 37
ZWBD : 59
L4z2 : 55
CUac : 41
xOGz : 68
4iHv : 24
sFhw : 21
QpfM : 57
pjeC : 36
wf9Q : 6
KQr0 : 78
eXGu : 3
29cg : 1
k5Eh : 9
OTVU : 46
iCka : 80
MbYr : 12
HaRb : 2
s78h : 1
rpo3 : 16
Yxep : 13
gutG : 14
hKDs : 6


In [28]:
len(result)

16160

In [17]:
df_c = pd.DataFrame(result)
df = df_c.drop_duplicates(subset=['vacancies','company','post_time','job','work_content'])
# _df = df_c[~df_c['year_salary'].isna()].reset_index(drop='index')

In [25]:
_df = df[~df['month_salary'].isna()].reset_index(drop='index')
_df.shape

(297, 18)

In [50]:
# duplicates = df_c.duplicated(subset=['company','vacancies','job','post_time', 'location','seniority','work_content','frequency_overtime'])
# df_c[duplicates]
# df_c = df_c.drop_duplicates(subset='work_content')

In [23]:
df.to_csv('Salary.csv', index=False,encoding='utf-8-sig')

In [52]:
df = pd.DataFrame(result)
_df = df[~df['year_salary'].isna()]

In [60]:
df_c

,vacancies,company,post_time,job,location,month_salary,year_salary,year-end_salary,bonus,dividends,other,mood,frequency_overtime,average_day,seniority,total_seniority,work_content,advise_content
0,倉管人員,裕隆集團_裕隆汽車製造股份有限公司,2023.05.01,全職,苗栗,30-31k,500-510k,140k,None,None,None,想換工作了,很少加班,8 hr,1,1,這裡的工作基本上都是跟著輸送帶作業的(汽車組裝也是輸送帶)有夠操的啦\n我被分到的是撿料人員...,體力好再來 雖然環境不會有噪音或是粉塵 可是工作都是跟著輸送帶的 不能慢的那種 也就是你要一...
1,生產技術工程師,裕隆集團_裕隆汽車製造股份有限公司,2023.03.06,全職,苗栗,45-46k,720-730k,120k,38k,18k,12k,想換工作了,常常加班,10 hr,3,5,公司算是老公司，所以做法非常老派，但是非常穩定可以穩穩的待\n1.因為要解析車子的問題但是因...,如果想進裕隆的朋友是非常喜歡寫報告及報告的人那這一間你會非常適合同一份報告，你可以寫三次分不...
2,工程師,裕隆集團_裕隆汽車製造股份有限公司,2023.02.28,全職,苗栗,43-44k,680-690k,86k,86k,None,None,有苦說不出,常常加班,9 hr,1,10,報告很多，實用價值低\n會議很多，有開等於沒開，開完一堆追蹤\n對主要專精車輛事業不如中華汽...,想學報告製作的可以來\n抗壓、奴性強可來\n不在意工作樂趣可來\n不追求成就感可來\n想比傳...
3,品檢技術員,裕隆集團_裕隆汽車製造股份有限公司,2022.11.23,全職,苗栗,31-32k,560-570k,150k,40k,None,None,有苦說不出,很少加班,9 hr,8,10,組車零件進料檢出，關係企業供應商不好搞，常搬出集團的長官壓你，廠商送來零件沒有在出貨檢卻要你...,傳產薪資待遇，科技業不人性管理，除了苗栗三義、銅鑼附近的人推薦，其他要三思，學車廠經驗OK不...
4,出納專員,裕隆集團_裕隆日產汽車股份有限公司,2022.11.03,全職,臺中,32-33k,380-390k,None,None,None,None,想換工作了,賣肝拼經濟,10 hr,1,1,1. 薪資，面試時說碩士高於32000，因為32000在公司是大學畢業生的價碼，結果拿到薪資...,"如果你夠奴再來,基本上每天都要加班,沒加班主管還會跟同事私底下抱怨你,請假也會被講壞話,就是..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,PM,匯豐汽車股份有限公司,2023.03.17,全職,臺北,34-35k,610-620k,200k,2000,None,None,平常心,很少加班,8 hr,3,4,傳統產業，部門氣氛融洽，同事人都很好，主管為三年左右論調一次，無空降主管，大公司制度流程較s...,較建議新鮮人或養老的人來～這邊容錯率高、注重私生活，下班基本上不吵你，員工旅遊也會去國外，我...
82,客服專員,匯豐汽車股份有限公司,2023.03.16,全職,臺北,33-34k,450-460k,60k,3000,None,None,非常開心,很少加班,8 hr,2,2,電話客服，一直打電話，開發陌生或潛在客戶，公司福利升遷不錯，還有完整教育訓練，但現在人怕被詐...,要敢開口開發客戶，熟汽車常識、保險、借貸等基礎法律問題，不怕被客戶拒絕。說很多電話要保護喉嚨...
83,車貸徵審專員,匯豐汽車股份有限公司,2023.03.09,全職,臺北,34-35k,650-660k,100k,22k,None,120k,有苦說不出,常常加班,10 hr,1,1,車貸審核，檢視客戶財力資料。辦公室在北投營業所樓上，不過我在職期間廁所走廊要自行打掃😅\n福...,新鮮人想做內勤行政性質的很適合來這邊，可以賺到很不錯的薪水，當時的我剛畢業薪資就贏過很多人了...
84,技師,匯豐汽車股份有限公司,2022.12.12,全職,新竹,30-31k,360-370k,None,None,None,None,非常開心,準時上下班,8 hr,1,1,每天上班被廠長釘業績，業績沒達到，廠長就會說：業績沒達到，是公司在養你你知道嗎？\n講真的每...,建議去別廠牌車子賣得比較好的保養廠做，不然匯豐商用客戶已經慢慢流向豐田，中華產品也沒在創新，...


In [68]:
df.to_csv('salary_data.csv', encoding='utf-8-sig')
_df.to_csv('salary_vip_data.csv', encoding='utf-8-sig')

## 參考code

In [38]:
# 存放每間公司爬取資訊
df_interview = pd.DataFrame(columns=['company_name', 'vacancies', 'post_time', 'interview_time', 'rank', 'status', 'difficult', 'process', 'question', 'suggestion'])

# 不同公司
for c in company_code:
    # 不同頁面
    for page_num in range(1, 1000):
        # 設置網頁網址
        url = f'https://salary.tw/c/{c}?page={page_num}&sort=newest'
        # url = f'https://interview.tw/c/0U2N?page=1&sort=newest'
        driver.get(url)
        # 獲取網頁內容
        pageSource = driver.page_source
        soup = BeautifulSoup(pageSource, 'html.parser')
        # 公司名稱
        _company_name = soup.find('h1', {'class': 'fz-h2 fz-tit'}).text
        # 取得所以有面試文章區塊
        content = soup.find_all('div', {'class':'iw_experience-item'})
        if (len(content)!=0):
        # 跟去每個面試文章取得職業名稱
            for i in content:
                # 職缺
                _vacancies = i.find('a', {'class':'fz-s1 fz-tit'}).text
                # 分享時間
                _post_time = i.find_all('span', {'class':'fz-s4 cl-basic-500'})[1].text[:-3]
                # 面試時間
                _interview_time = i.find_all('span', {'class':'fz-s2'})[1].text
                # 評價（幾顆星）
                _rank = len(i.find_all('li', {'class':'iw_rate'}))
                # 面試狀態
                _status = i.find_all('span', {'class':'fz-s2'})[2].text
                # 難易度
                _difficult = i.find_all('span', {'class':'fz-s2'})[3].text
                # 面試過程
                try:
                    _process_div = i.find_all('div', {'class':'iw_info-wrap__con cl-basic-700'})
                    _process = _process_div[1].find('span').text
                    # _process = i.find_all('span', {'class':'fz-t3'})[0].text
                except:
                    print("面試過程")
                    print(_company_name)
                    print("page: ",page_num)
                    print(_vacancies)
                    _process = None
                # 面試問題
                try:
                    _question = []
                    _Q = []
                    _A = []
                    ans_title = i.find_all('div', {'class':'ans-title'})
                    for q in ans_title:
                        _Q.append(q.find('span').text)
                    ans_content = i.find_all('div', {'class':'ans-content'})
                    for q in ans_content:
                        _A.append(q.find('span').text)
                    for k in range(0, len(_Q)):
                        tmp = {
                            "Q": _Q[k],
                            "A": _A[k]
                        }
                        _question.append(tmp)
                except:
                    print("面試問題")
                    print(_company_name)
                    print("page: ",page_num)
                    print(_vacancies)
                    _question = None
                # 面試建議
                try:
                    _suggestion_div = i.find_all('div', {'class':'iw_info-wrap__con cl-basic-700'})
                    _suggestion = _suggestion_div[2].find('span').text
                    # _suggestion = i.find_all('span', {'class':'fz-t3'})[1].text
                except Exception as e:
                    _suggestion = None
                tmp_df_interview = pd.DataFrame({
                    "company_name": [_company_name],
                    "vacancies": [_vacancies],
                    "post_time": [_post_time],
                    "interview_time": [_interview_time],
                    "rank": [_rank],
                    "status": [_status],
                    "difficult": [_difficult],
                    "process": [_process],
                    "question": [_question],
                    "suggestion": [_suggestion]
                }, index=[len(df_interview)])
                df_interview = df_interview.append(tmp_df_interview)
        else:
            break
        time.sleep(random.uniform(1.0, 2.0)) 
    # 將爬取到的信息保存到 MySQL 資料庫中
#     sql = "INSERT INTO your_table (title, link, author, date, content) VALUES (%s, %s, %s, %s, %s)"
#     # cursor.execute(sql, (title, link, author, date, content))
#     # db.commit()

面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
企劃管理師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
管理師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
行銷專員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
人力資源管理師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
秘書
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
人資
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
資深人資管理師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
行銷
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
業務
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
行銷
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
材質規劃師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
DMS-工程師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
銷售顧問
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
財務人員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
技師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  2
業務
面試過程
裕隆集團_納智捷汽車股份有限公司
page:  2
行政人員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  2
行政人員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  2
服務專員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  2
行政專員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  2
企劃人員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  2
人資專員
面試建議
Luxgen_北智捷汽車股份有限公司
page:  1
行政人員
面試建議
Luxgen_北智捷汽車股份有限公司
page:  1
企劃專員
面試建議
Luxgen_北智捷汽車股份有限公司
page:  1
汽車銷售顧問
面試建議
Luxgen_北智捷汽車股份有限公司
page:  1
業務
面試建議
Luxgen_北智捷汽車股份有限公司
page:  1
業務
面試建議
Luxgen_北智捷汽車股份有限公司


In [40]:
df_interview

,company_name,vacancies,post_time,interview_time,rank,status,difficult,process,question,suggestion
0,裕隆集團_納智捷汽車股份有限公司,企劃管理師,2022.08.24,2022.07,3,未錄取,普通,針對專業提出疑問，並...,"[{'Q': '這個品牌業界傳的都不太好，為什麼願意來納智捷面試', 'A': '因為我本來...",None
1,裕隆集團_納智捷汽車股份有限公司,管理師,2022.02.21,2019.02,1,無聲卡,低,整體面試過程約2小時...,"[{'Q': '對於納智捷的了解', 'A': '依照職位跟公司官網做...'}]",None
2,裕隆集團_納智捷汽車股份有限公司,行銷專員,2022.01.13,2017.09,3,錄取,普通,第一次面試先簡單自我...,"[{'Q': '會不會開車', 'A': '有駕照，但當時還不熟...'}]",None
3,裕隆集團_納智捷汽車股份有限公司,人力資源管理師,2021.09.12,2021.09,5,未錄取,超高,第一關，人資面試\n...,"[{'Q': '結算薪資怎麼做', 'A': '照実回答'}]",None
4,裕隆集團_納智捷汽車股份有限公司,秘書,2021.04.21,2017.09,5,錄取,普通,在台南東區的廠二樓面...,"[{'Q': '與人交談方面的問題', 'A': '我是個開朗的人，比較...'}]",None
...,...,...,...,...,...,...,...,...,...,...
7608,匯豐汽車股份有限公司,產品企劃專員,2018.04.20,2017.12,4,錄取,簡單,簡報說明令人印象深刻,"[{'Q': '針對產品提出看法', 'A': '直接腦袋中的想法講出...'}]",None
7609,匯豐汽車股份有限公司,SUM服務專員,2018.03.23,2010.06,5,錄取,簡單,面試過程其實很好，而...,"[{'Q': '時間有點久不太記得，但是要看你遇到的廠長是好是壞!', 'A': '時間有點...",None
7610,匯豐汽車股份有限公司,總公司-品牌部,2018.01.24,2016.06,3,錄取,簡單,帶到會議室，等待面試...,[{'Q': '對於行銷與品牌的各種看法，EXCEL操作技能，網站經營(GA)，網路行銷規劃...,None
7611,匯豐汽車股份有限公司,技師,2017.12.24,2002.06,4,錄取,簡單,直接面試 問維修相關...,"[{'Q': '需配合值班可否接受 前份工作為何離職', 'A': '可以 職涯規劃'}]",None


In [41]:
df_interview.to_csv('interview_test.csv', encoding='utf-8')